# Etapa 04 – Otimização de Hiperparâmetros com Optuna

Após aplicar os modelos com balanceamento de classes e thresholds dinâmicos, partimos para a **otimização de hiperparâmetros** para extrair o máximo desempenho de cada algoritmo.

---

## 🎯 Objetivo

- Identificar os melhores hiperparâmetros para os modelos:
  - Regressão Logística
  - Random Forest
  - LightGBM

- Utilizar a métrica **F1-Score da classe positiva (desligamento)** como critério de avaliação, priorizando o equilíbrio entre precisão e recall.

---

## ⚙️ Metodologia

- Utilizamos o framework **Optuna** para busca automática dos melhores hiperparâmetros.
- Cada modelo teve uma função `objective` separada.
- A base de dados foi preparada pela função `preparar_dados_rh()`.
- Realizamos **30 iterações (trials)** por modelo para obter uma boa cobertura do espaço de busca.

---

## 🔍 Hiperparâmetros otimizados

### 🔹 Regressão Logística
- `C`: controle da regularização (loguniform entre 1e-4 e 1e2)
- `solver`: escolha entre `'liblinear'` e `'lbfgs'`
- `class_weight='balanced'` fixado

### 🔹 Random Forest
- `n_estimators`: número de árvores (entre 50 e 200)
- `max_depth`: profundidade máxima da árvore (entre 2 e 20)
- `class_weight='balanced'` fixado

### 🔹 LightGBM
- `num_leaves`: número de folhas por árvore (entre 20 e 150)
- `learning_rate`: taxa de aprendizado (loguniform entre 1e-3 e 0.2)
- `n_estimators`: número de iterações (entre 50 e 300)
- `is_unbalance=True` fixado

---

## 📈 Métrica de Avaliação
A métrica utilizada durante a otimização foi o **F1-Score da classe 1** no conjunto de teste. Essa escolha se justifica pois:

- O dataset é desbalanceado (apenas ~20% de desligamentos).
- Precisamos capturar tanto **quantos casos de desligamento conseguimos prever** (`recall`) quanto **com que precisão os prevemos** (`precision`).
- O F1-Score reflete o equilíbrio entre os dois.

---

## 📌 Resultados (exemplo)

```text
✅ Logistic Regression
Melhor F1-Score: 0.38
Melhores Hiperparâmetros: {'C': 0.47, 'solver': 'liblinear'}

✅ Random Forest
Melhor F1-Score: 0.41
Melhores Hiperparâmetros: {'n_estimators': 180, 'max_depth': 11}

✅ LightGBM
Melhor F1-Score: 0.44
Melhores Hiperparâmetros: {'num_leaves': 42, 'learning_rate': 0.07, 'n_estimators': 220}
```
---

## 🧠 Conclusão
```text
A introdução da função genérica `tunar_modelo_optuna()` tornou o processo de otimização mais limpo, modular e reutilizável. Com isso, foi possível aplicar o mesmo framework de tuning para múltiplos algoritmos com pouquíssimas linhas de código, mantendo consistência nos critérios de validação cruzada e avaliação.

O uso do Optuna permitiu uma busca eficiente e orientada por F1-Score para a classe de desligamento, garantindo modelos mais ajustados à realidade desbalanceada do problema.

A próxima etapa será consolidar os melhores modelos, avaliar com métricas complementares (AUC, curva de precisão-recall) e aplicar técnicas de interpretabilidade para guiar a tomada de decisão no RH.

```

In [1]:
from sklearn.metrics import classification_report
import sys
import os
sys.path.append(os.path.abspath("../"))
from scripts.func import preparar_dados_rh, tunar_modelo_optuna

# Preparar dados
df_encoded, X_train, X_test, y_train, y_test = preparar_dados_rh(qtd_amostras=100_000, semente=42)

# Tunar modelos
model_lgbm, study_lgbm = tunar_modelo_optuna(modelo_nome="lightgbm", X_train=X_train, y_train=y_train, n_trials=30)
model_rf, study_rf = tunar_modelo_optuna(modelo_nome="random_forest", X_train=X_train, y_train=y_train, n_trials=30)
model_lr, study_lr = tunar_modelo_optuna(modelo_nome="logistic_regression", X_train=X_train, y_train=y_train, n_trials=30)

print("\n📌 Avaliação no conjunto de teste:")
for nome, modelo in zip(["LightGBM", "Random Forest", "Logistic Regression"],
                        [model_lgbm, model_rf, model_lr]):
    print(f"\n📍 {nome}")
    y_pred = modelo.predict(X_test)
    print(classification_report(y_test, y_pred, zero_division=0))

/Users/drico/Desktop/Mack/hr-analytics/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-07-02 03:28:11,831] A new study created in memory with name: Tuning_lightgbm
Best trial: 0. Best value: 0.624425:   3%|▎         | 1/30 [01:20<38:57, 80.61s/it]

[I 2025-07-02 03:29:32,453] Trial 0 finished with value: 0.6244254275568708 and parameters: {'learning_rate': 0.008468008575248327, 'num_leaves': 286, 'max_depth': 12, 'min_child_samples': 64, 'feature_fraction': 0.6624074561769746, 'bagging_fraction': 0.662397808134481, 'bagging_freq': 1, 'lambda_l1': 4.330880728874676, 'lambda_l2': 3.005575058716044}. Best is trial 0 with value: 0.6244254275568708.


Best trial: 0. Best value: 0.624425:   7%|▋         | 2/30 [01:31<18:27, 39.54s/it]

[I 2025-07-02 03:29:43,239] Trial 1 finished with value: 0.6190044641435256 and parameters: {'learning_rate': 0.05675206026988745, 'num_leaves': 15, 'max_depth': 15, 'min_child_samples': 85, 'feature_fraction': 0.6849356442713105, 'bagging_fraction': 0.6727299868828402, 'bagging_freq': 2, 'lambda_l1': 1.5212112147976886, 'lambda_l2': 2.6237821581611893}. Best is trial 0 with value: 0.6244254275568708.


Best trial: 0. Best value: 0.624425:  10%|█         | 3/30 [02:21<19:57, 44.35s/it]

...

Best trial: 26. Best value: 0.613533:  97%|█████████▋| 29/30 [37:05<01:04, 64.13s/it]

[I 2025-07-02 04:22:50,588] Trial 28 finished with value: 0.6135191484085669 and parameters: {'C': 0.15927583972309522, 'penalty': 'l1'}. Best is trial 26 with value: 0.6135328250736571.


Best trial: 26. Best value: 0.613533: 100%|██████████| 30/30 [37:12<00:00, 74.42s/it]


[I 2025-07-02 04:22:57,234] Trial 29 finished with value: 0.6130713781954892 and parameters: {'C': 0.034089686069196455, 'penalty': 'l1'}. Best is trial 26 with value: 0.6135328250736571.

✅ Melhor AUC: 0.6135
🏆 Melhor conjunto de hiperparâmetros:
   - C: 0.20741339470550815
   - penalty: l1


ValueError: Solver lbfgs supports only 'l2' or None penalties, got l1 penalty.


# Etapa 04.5 – Treinamento Final e Salvamento do Melhor Modelo

Antes de partirmos para a interpretabilidade na Etapa 05, é necessário garantir que o modelo treinado com os melhores hiperparâmetros esteja disponível em disco, assim como o preprocessador utilizado. Isso permite que a aplicação posterior (via API ou script) utilize os mesmos padrões de transformação e predição.

---

## 🎯 Objetivo

- Treinar o modelo LightGBM com os **melhores hiperparâmetros** identificados na otimização com Optuna.
- Salvar o modelo final em um arquivo `.pkl` para uso posterior.
- Salvar o **preprocessador de dados** utilizado, garantindo que novos dados sejam transformados da mesma forma.

---

## 🧪 Pipeline Executado

### 1. Preparação dos Dados

```python
from scripts.func import preparar_dados_rh

df_encoded, X_train, X_test, y_train, y_test = preparar_dados_rh(qtd_amostras=100_000, semente=42)
```

---

### 2. Otimização e Treinamento com os Melhores Parâmetros

```python
from scripts.func import tunar_modelo_optuna

modelo_lgbm, estudo_lgbm = tunar_modelo_optuna(
    modelo_nome='lightgbm',
    X_train=X_train,
    y_train=y_train,
    n_trials=30,
    semente=42
)
```

---

### 3. Criação do Preprocessador

```python
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

colunas_numericas = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
colunas_categoricas = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), colunas_numericas),
    ('cat', OneHotEncoder(drop='first', sparse_output=False), colunas_categoricas)
])

preprocessor.fit(X_train)
```

---

### 4. Salvamento dos Arquivos

```python
import joblib
import os

os.makedirs('models', exist_ok=True)

joblib.dump(modelo_lgbm, 'models/lightgbm_model.pkl')
joblib.dump(preprocessor, 'models/preprocessor.pkl')
```

---

## 💾 Resultado

- `models/lightgbm_model.pkl`: Modelo final com os hiperparâmetros otimizados.
- `models/preprocessor.pkl`: Pipeline de transformação de dados usado no treinamento.

---

## ✅ Próximo Passo

Com o modelo e preprocessador salvos, estamos prontos para seguir para a **Etapa 05 – Aplicação e Interpretabilidade**, onde iremos:

- Gerar gráficos de ROC, Precision-Recall e matriz de confusão.
- Visualizar a importância das variáveis.
- Aplicar técnicas de interpretabilidade com SHAP.
- Preparar uma função final para uso em API ou serviço de predição.


In [2]:
import joblib
import os
import sys
import os
sys.path.append(os.path.abspath("../"))
from scripts.func import preparar_dados_rh, tunar_modelo_optuna

# 1. Gerar dados
df_encoded, X_train, X_test, y_train, y_test = preparar_dados_rh(qtd_amostras=100_000, semente=42)

# 2. Treinar o melhor modelo
modelo_lgbm, estudo_lgbm = tunar_modelo_optuna(
    modelo_nome='lightgbm',
    X_train=X_train,
    y_train=y_train,
    n_trials=30,
    semente=42
)

# 3. Criar e treinar o preprocessor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

colunas_numericas = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
colunas_categoricas = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), colunas_numericas),
    ('cat', OneHotEncoder(drop='first', sparse_output=False), colunas_categoricas)
])

preprocessor.fit(X_train)

# 4. Salvar arquivos

root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
models_dir = os.path.join(root_dir, 'models')
if not os.path.exists(models_dir):
    os.makedirs(models_dir, exist_ok=True)

joblib.dump(modelo_lgbm, os.path.join(models_dir, 'lightgbm_model.pkl'))
joblib.dump(preprocessor, os.path.join(models_dir, 'preprocessor.pkl'))


[I 2025-07-02 04:51:23,213] A new study created in memory with name: Tuning_lightgbm
Best trial: 0. Best value: 0.624425:   3%|▎         | 1/30 [01:19<38:27, 79.58s/it]

[I 2025-07-02 04:52:42,780] Trial 0 finished with value: 0.6244254275568708 and parameters: {'learning_rate': 0.008468008575248327, 'num_leaves': 286, 'max_depth': 12, 'min_child_samples': 64, 'feature_fraction': 0.6624074561769746, 'bagging_fraction': 0.662397808134481, 'bagging_freq': 1, 'lambda_l1': 4.330880728874676, 'lambda_l2': 3.005575058716044}. Best is trial 0 with value: 0.6244254275568708.


Best trial: 0. Best value: 0.624425:   7%|▋         | 2/30 [01:30<18:13, 39.04s/it]

[I 2025-07-02 04:52:53,447] Trial 1 finished with value: 0.6190044641435256 and parameters: {'learning_rate': 0.05675206026988745, 'num_leaves': 15, 'max_depth': 15, 'min_child_samples': 85, 'feature_fraction': 0.6849356442713105, 'bagging_fraction': 0.6727299868828402, 'bagging_freq': 2, 'lambda_l1': 1.5212112147976886, 'lambda_l2': 2.6237821581611893}. Best is trial 0 with value: 0.6244254275568708.


Best trial: 0. Best value: 0.624425:  10%|█         | 3/30 [02:14<18:44, 41.65s/it]

...

Best trial: 28. Best value: 0.641066:  97%|█████████▋| 29/30 [09:59<00:08,  8.09s/it]

[I 2025-07-02 05:01:23,178] Trial 28 finished with value: 0.6410656170156999 and parameters: {'learning_rate': 0.0057895274890742195, 'num_leaves': 73, 'max_depth': 3, 'min_child_samples': 70, 'feature_fraction': 0.9551883483386383, 'bagging_fraction': 0.9128563041504967, 'bagging_freq': 9, 'lambda_l1': 0.39829367118956727, 'lambda_l2': 4.781042957486154}. Best is trial 28 with value: 0.6410656170156999.


Best trial: 28. Best value: 0.641066: 100%|██████████| 30/30 [10:05<00:00, 20.20s/it]


[I 2025-07-02 05:01:29,176] Trial 29 finished with value: 0.636315280520333 and parameters: {'learning_rate': 0.022477924168153415, 'num_leaves': 77, 'max_depth': 3, 'min_child_samples': 67, 'feature_fraction': 0.9443130856149027, 'bagging_fraction': 0.8485281092713779, 'bagging_freq': 10, 'lambda_l1': 0.05174219484548148, 'lambda_l2': 4.9939983310912455}. Best is trial 28 with value: 0.6410656170156999.

✅ Melhor AUC: 0.6411
🏆 Melhor conjunto de hiperparâmetros:
   - learning_rate: 0.0057895274890742195
   - num_leaves: 73
   - max_depth: 3
   - min_child_samples: 70
   - feature_fraction: 0.9551883483386383
   - bagging_fraction: 0.9128563041504967
   - bagging_freq: 9
   - lambda_l1: 0.39829367118956727
   - lambda_l2: 4.781042957486154


['models/preprocessor.pkl']